<a href="https://colab.research.google.com/github/DonghaeSuh/PyTorch_Basic/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 경로 이동

In [1]:
cd drive/MyDrive/pytorch

/content/drive/MyDrive/pytorch


In [2]:
ls

dataset.csv  Model.ipynb


# 메인 코드

In [3]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset,DataLoader

In [ ]:
df=pd.read_csv('dataset.csv')
df.head()

,x,y
0,-10.0,327.79
1,-9.9,321.39
2,-9.8,314.48
3,-9.7,308.51
4,-9.6,302.86


In [ ]:
a=df['x'].values

In [ ]:
b=df.iloc[:,0].values

In [ ]:
a==b

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

### 사용자 정의 dataset

In [12]:
class CustomDataset(Dataset):
  def __init__(self,file_path):
    df=pd.read_csv(file_path)
    self.x=df.iloc[:,0].values
    self.y=df.iloc[:,1].values
    self.length=len(df)

  def __getitem__(self, index):
    x=torch.FloatTensor([self.x[index]**2, self.x[index]])
    y=torch.FloatTensor([self.y[index]])
    return x,y

  def __len__(self):
    return self.length

image.png

### 사용자 정의 모델

In [13]:
class CustomModel(nn.Module):
  def __init__(self):
    super(CustomModel,self).__init__()
    self.layer=nn.Linear(2,1)
  
  def forward(self,x):
    x=self.layer(x)
    return x

### 데이터 사용 준비

In [14]:
train_dataset=CustomDataset("./dataset.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)
# batch 크기 : 128 / 데이터 순서 셔플 / 마지막 batch를 제거

### GPU 연산 사용

In [15]:
device="cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr =0.0001)

### 학습

In [20]:
for epoch in range(10000):
  cost=0.0

  for x, y in train_dataloader:
    x=x.to(device)
    y=y.to(device)

    output=model(x)
    loss=criterion(output,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost+=loss

  cost=cost/len(train_dataloader)

  if (epoch+1)%1000==0:
    print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")


Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1025, -1.7032]], requires_grad=True), Parameter containing:
tensor([0.3361], requires_grad=True)], Cost : 0.087
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1022, -1.7031]], requires_grad=True), Parameter containing:
tensor([0.3495], requires_grad=True)], Cost : 0.081
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1019, -1.7031]], requires_grad=True), Parameter containing:
tensor([0.3619], requires_grad=True)], Cost : 0.085
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1020, -1.7034]], requires_grad=True), Parameter containing:
tensor([0.3732], requires_grad=True)], Cost : 0.074
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1022, -1.7030]], requires_grad=True), Parameter containing:
tensor([0.3834], requires_grad=True)], Cost : 0.083
Epoch : 6000, Model : [Parameter containing:
tensor([[ 3.1018, -1.7033]], requires_grad=True), Parameter containing:
tensor([0.3929], requires_grad=True)],

### 모델 평가

이 모델은 실제로 y=3.1x**2 -1.7x + random(0.01,0.99)로 구성되어 있음.

In [22]:
with torch.no_grad():
  model.eval() # 평가(Inference)모드 => 다시 학습을 진행하려면 model.train()을 호출해야한다.
  test_inputs=torch.FloatTensor([[1**2,1],[5**2,5],[11**2,11]]).to(device)
  outputs=model(test_inputs)
  print(outputs)

tensor([[  1.8210],
        [ 69.4368],
        [356.9348]])
